In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

## Original optimization problem

In [2]:
# name the variables, 12 is more understandable
var_names = ["A","A_over", # grams of environment A and its overtime
             "B","B_over", # grams of environment B and its overtime
             "A_Ag1","A_Ag1_over", # grams of mix produced by A and baked by Agent1 and its overtime
             "A_Ag2","A_Ag2_over", # grams of mix produced by A and baked by Agent2 and its overtime
             "B_Ag1","B_Ag1_over", # grams of mix produced by B and baked by Agent1 and its overtime
             "B_Ag2","B_Ag2_over" # grams of mix produced by B and baked by Agent2 and its overtime
            ]

In [3]:
# name the objective function
Profit = np.array([33,29,
                   54.5,50.5,
                   -18,-24.5,
                   -12.5,-19,
                   -18,-24.5,
                   -12.5,-19,])

In [4]:
# name constraints matrix A
A = np.array([
    [1,0,1,0,0,0,0,0,0,0,0,0], # total of grams A&B <=1000
    [0,1,0,1,0,0,0,0,0,0,0,0], # total of grams A&B overtime <=400
    [0,0,0,0,1,0,1,0,1,0,1,0], # total work load of Agents <=800
    [0,0,0,0,0,1,0,1,0,1,0,1], # total overtime work of Agents <=300
    
    # for == constraints, use <= & >=
    [0.44,0.44,0,0,-1,-1,-1,-1,0,0,0,0], # total mix produced by A <= total product baked from mix produced by A
    [-0.44,-0.44,0,0,1,1,1,1,0,0,0,0], # total mix produced by A >= total product baked from mix produced by A    
    [0,0,0.72,0.72,0,0,0,0,-1,-1,-1,-1], # total mix produced by B <= total product baked from mix produced by B
    [0,0,-0.72,-0.72,0,0,0,0,1,1,1,1], # total mix produced by B >= total product baked from mix produced by B
    
    [0,0,0,0,-0.1,-0.1,0,0,-0.2,-0.2,0.1,0.1], # % good in product >= 40%
    [0,0,0,0,-0.02,-0.02,-0.01,-0.01,0.03,0.03,-0.01,-0.01] # % bad in product <= 7%
    ])

In [5]:
# name constraints vector b
rhs =np.array([1000,400,800,300,0,0,0,0,0,0])

In [6]:
# name linear optimization problem
m = gp.Model("critters")
x = m.addMVar(12,obj=Profit,name=var_names)
m.setObjective(Profit @ x, GRB.MAXIMIZE)
m.addConstr(A @ x <= rhs)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-16


<MConstr (10,) *awaiting model update*>

In [7]:
# solve optimization
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 12 columns and 50 nonzeros
Model fingerprint: 0x0d237aac
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 8 rows, 12 columns, 38 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.4700000e+04   1.260000e+02   0.000000e+00      0s
       5    5.5656813e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.565681250e+04


In [8]:
print("\n\n Total Profit = {0:,.2f} ".format(m.ObjVal))



 Total Profit = 55,656.81 


In [10]:
m.printAttr(['X', 'Obj'])


    Variable            X          Obj 
--------------------------------------
           A      196.875           33 
      A_over            0           29 
           B      803.125         54.5 
      B_over          400         50.5 
       A_Ag1       86.625          -18 
  A_Ag1_over            0        -24.5 
       A_Ag2            0        -12.5 
  A_Ag2_over            0          -19 
       B_Ag1      259.875          -18 
  B_Ag1_over            0        -24.5 
       B_Ag2        453.5        -12.5 
  B_Ag2_over      152.875          -19 


In [11]:
m.printAttr(['RHS', 'Sense', 'Slack'])


  Constraint          RHS        Sense        Slack 
---------------------------------------------------
          R0         1000            <            0 
          R1          400            <            0 
          R2          800            <            0 
          R3          300            <      147.125 
          R4           -0            <            0 
          R5           -0            <            0 
          R6           -0            <            0 
          R7           -0            <            0 
          R8           -0            <            0 
          R9           -0            <            0 


## Sensitivity analysis

In [12]:
def solve_print():
    m.reset()
    m.optimize()
    print("Total Profit = {0:.2f} ".format(m.ObjVal))
    m.printAttr(['X', 'Obj', 'rc', 'saobjlow', 'saobjup'])
    m.printAttr(['RHS', 'Sense', 'Slack', 'pi', 'sarhslow', 'sarhsup' ])

In [13]:
def set_obj (i, j):
    m.getVars()[i].obj = j

In [14]:
def set_rhs(i,j):
    m.getConstrs()[i].rhs=j

In [15]:
def new_obj_calc(i,j):
    save = m.getVars()[i].obj
    if j>0: new = objup[i] + epsilon
    else: new = objlow[i] - epsilon
    print("\n\n New Obj ",i," = ",new)
    set_obj(i,new)
    solve_print()
    set_obj(i,save)
    m.update()

In [16]:
def new_rhs_calc(i,j):
    save = m.getConstrs()[i].rhs
    if j>0: new = rhsup[i] + epsilon
    else: new = rhslow[i] - epsilon 
    print("\n\n New RHS ",i," = ",new)
    set_rhs(i,new)
    solve_print()
    set_rhs(i,save)
    m.update()

In [17]:
# name linear optimization problem
m = gp.Model("critters")
x = m.addMVar(12,obj=Profit,name=var_names)
m.setObjective(Profit @ x, GRB.MAXIMIZE)
m.addConstr(A @ x <= rhs)
m.update()

# Solve original problem
solve_print()

Discarded solution information
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 12 columns and 50 nonzeros
Model fingerprint: 0x0d237aac
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 38 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.4700000e+04   1.260000e+02   0.000000e+00      0s
       5    5.5656813e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.565681250e+04
Total Profit = 55656.81 

    Variable            X          Obj           rc     saobjlow      saobjup 
-------

In [18]:
# save the original problem details before sensitivity
objup = m.getAttr('SaObjUp')
objlow = m.getAttr('SaObjlow')
rhsup = m.getAttr('SaRhsUp')
rhslow = m.getAttr('SaRhsLow')

In [19]:
# # name the variables, 12 is more understandable
# var_names = ["A","A_over",
#              "B","B_over",
#              "A_Ag1","A_Ag1_over",
#              "A_Ag2","A_Ag2_over",
#              "B_Ag1","B_Ag1_over",
#              "B_Ag2","B_Ag2_over"]
epsilon = 0.1
# Check the A sensitivity
new_obj_calc(0,1)
new_obj_calc(0,-1)
# a small increase in A coef will cause change in the result



 New Obj  0  =  37.96800000000001
Discarded solution information
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 12 columns and 50 nonzeros
Model fingerprint: 0x03f167bf
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 38 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.4700000e+04   1.260000e+02   0.000000e+00      0s
       7    5.6634887e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.663488750e+04
Total Profit = 56634.89 

    Variable            X          Obj           r

In [20]:
# Check the A_over sensitivity
new_obj_calc(1,1)
new_obj_calc(1,-1)



 New Obj  1  =  29.099999999999998
Discarded solution information
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 12 columns and 50 nonzeros
Model fingerprint: 0x1426a93b
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 38 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.4700000e+04   1.260000e+02   0.000000e+00      0s
       5    5.5676500e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.567650000e+04
Total Profit = 55676.50 

    Variable            X          Obj           

In [21]:
# can check other vector c elements sensitivity
# < saobjlow | > saobjup, will cause changes in the optimized x vector
# similar for constraints

## e)

In [22]:
# will change the A matrix
# will change the obj function
obj=[]
Profit_new=Profit.copy()
A_new=A.copy()
for i in [0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10]:
    for j in [0,0.01,0.02,0.03,0.04,0.05]:
        Profit_new[0],Profit_new[1]=Profit[0]+44*i,Profit[1]+44*i
        Profit_new[2],Profit_new[3]=Profit[2]+72*j,Profit[3]+72*j
        
        A_new[4][0],A_new[4][1]=A[4][0]+0.44*i,A[4][1]+0.44*i
        A_new[5][0],A_new[5][1]=A[5][0]-0.44*i,A[5][1]-0.44*i
        
        A_new[6][2],A_new[6][3]=A[6][2]+0.72*j,A[6][3]+0.72*j
        A_new[6][2],A_new[6][3]=A[6][2]-0.72*j,A[6][3]-0.72*j
        
        m = gp.Model("critters")
        x = m.addMVar(12,obj=Profit_new,name=var_names)
        m.setObjective(Profit_new @ x, GRB.MAXIMIZE)
        m.addConstr(A_new @ x <= rhs)
        m.optimize()
        obj+=[m.ObjVal]
# obj

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 12 columns and 50 nonzeros
Model fingerprint: 0x0d237aac
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 38 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.4700000e+04   1.260000e+02   0.000000e+00      0s
       5    5.5656813e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.565681250e+04
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Th

In [24]:
obj_sublists = [obj[i:i+6] for i in range(0, len(obj), 6)]
tbl_e=pd.DataFrame(obj_sublists)
tbl_e.index=['A_yield+'+str(i)+'%' for i in range(0,11)]
tbl_e.columns=['B_yield+'+str(i)+'%' for i in range(0,6)]
display(tbl_e)

,B_yield+0%,B_yield+1%,B_yield+2%,B_yield+3%,B_yield+4%,B_yield+5%
A_yield+0%,55656.812500,56732.915663,57812.053902,58894.240075,59979.487115,61067.808025
A_yield+1%,55750.865376,56827.645461,57907.436582,58990.251388,60076.102595,61165.002995
A_yield+2%,55843.329032,56920.772386,58001.203105,59084.633633,60171.076478,61260.544223
A_yield+3%,55934.243412,57012.336779,58093.394202,59177.427923,60264.450250,61354.473558
A_yield+4%,56023.647130,57102.377637,58184.049246,59268.674000,60356.264010,61446.831452
A_yield+5%,56111.577528,57190.932673,58273.206309,59358.410289,60446.556530,61537.657014
A_yield+6%,56198.070728,57278.038363,58360.902217,59446.673955,60535.365306,61626.988063
A_yield+7%,56283.161680,57363.730000,58447.172596,59533.500953,60622.726614,61714.861187
A_yield+8%,56366.884211,57448.041739,58532.051926,59618.926077,60708.675556,61801.311788
A_yield+9%,56449.271066,57531.006645,58615.573582,59702.983005,60793.246106,61886.374131


## f)

In [25]:
# will change the A matrix
obj=[]
A_new=A.copy()
for i in [0.40,0.39,0.38,0.37,0.36,0.35]:
    for j in [0.07,0.08,0.09,0.10]:
        A_new[8]=np.repeat(np.array([0,0,i-0.5,i-0.4,i-0.6,i-0.3]),2)
        A_new[9]=np.repeat(np.array([0,0,0.05-j,0.06-j,0.1-j,0.06-j]),2)
        m = gp.Model("critters")
        x = m.addMVar(12,obj=Profit,name=var_names)
        m.setObjective(Profit @ x, GRB.MAXIMIZE)
        m.addConstr(A_new @ x <= rhs)
        m.optimize()
        obj+=[m.ObjVal]
# obj

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 12 columns and 50 nonzeros
Model fingerprint: 0x87b8571f
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 38 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.4700000e+04   1.260000e+02   0.000000e+00      0s
       5    5.5656813e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.565681250e+04
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Th

In [26]:
obj_sublists = [obj[i:i+4] for i in range(0, len(obj), 4)]
tbl_f=pd.DataFrame(obj_sublists)
tbl_f.index=['limit_good='+str(i)+'%' for i in [40,39,38,37,36,35]]
tbl_f.columns=['limit_bad='+str(i)+'%' for i in [7,8,9,10]]
display(tbl_f)

,limit_bad=7%,limit_bad=8%,limit_bad=9%,limit_bad=10%
limit_good=40%,55656.812500,58900.0,58900.0,58900.0
limit_good=39%,57089.169329,59084.8,59084.8,59084.8
limit_good=38%,58587.058824,59269.6,59269.6,59269.6
limit_good=37%,59454.400000,59454.4,59454.4,59454.4
limit_good=36%,59639.200000,59639.2,59639.2,59639.2
limit_good=35%,59824.000000,59824.0,59824.0,59824.0


## g)

In [27]:
# similar to add an more expensive option of overtime working
var_names_new=var_names+['A_newstuff','B_newstuff','A_Ag1_newstuff','A_Ag2_newstff','B_Ag1_newstuff','B_Ag2_newstff']

In [28]:
Profit_new=np.array([33,29,
                 54.5,50.5,
                 -18,-24.5,
                 -12.5,-19,
                 -18,-24.5,
                 -12.5,-19,
                 3,24.5,
                 -18-20,-12.5-20,
                 -18-20,-12.5-20])

In [29]:
# name constraints matrix A
A_new = np.array([
    [1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0],
    [0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0],
    [0.44,0.44,0,0,-1,-1,-1,-1,0,0,0,0,0.44,0,-1,-1,0,0],
    [-0.44,-0.44,0,0,1,1,1,1,0,0,0,0,-0.44,0,1,1,0,0],
    [0,0,0.72,0.72,0,0,0,0,-1,-1,-1,-1,0,0.72,0,0,-1,-1],
    [0,0,-0.72,-0.72,0,0,0,0,1,1,1,1,0,-0.72,0,0,1,1],
    [0,0,0,0,-0.1,-0.1,0,0,-0.2,-0.2,0.1,0.1,0,0,-0.1,0,-0.2,0.1],
    [0,0,0,0,-0.02,-0.02,-0.01,-0.01,0.03,0.03,-0.01,-0.01,0,0,-0.02,-0.01,0.03,-0.01]
    ])

In [30]:
# name linear optimization problem
m = gp.Model("critters")
x = m.addMVar(18,obj=Profit_new,name=var_names_new)
m.setObjective(Profit_new @ x, GRB.MAXIMIZE)
m.addConstr(A_new @ x <= rhs)

# solve optimization
m.optimize()

m.printAttr(['X', 'Obj'])

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 18 columns and 69 nonzeros
Model fingerprint: 0xb8ac9c7b
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [3e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 18 columns, 51 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4400000e+32   1.600000e+30   2.440000e+02      0s
       9    5.7209596e+04   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.720959596e+04

    Variable            X          Obj 
--------------------------------------
           A      227.273           33 
      A_over           

In [31]:
# should hire more stuff to harvest B environment